# 1. Data Reading and Visualization

In [ ]:
import pandas as pd
import os
import gc
print(os.listdir("../input"))

In [ ]:
train=pd.read_json('../input/cuisine/train.json').set_index('id')
test=pd.read_json('../input/cuisine/test.json').set_index('id')

In [ ]:
print(train.shape)
train.head()

In [ ]:
print(test.shape)
test.head()

In [ ]:
train.isnull().sum()

no missing value with around 40k training data points

In [ ]:
print('there are %s classes in total' % len(train.cuisine.unique()))
train.cuisine.unique().tolist()

# 2. GLOVE Embedding + Mean+ Random Forest

In [ ]:
import numpy as np
def loadGloveModel(gloveFile):
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    return model

In [ ]:
model=loadGloveModel('../input/glove6b/glove.6B.50d.txt')

In [ ]:
all=train['ingredients'].tolist()+test['ingredients'].tolist()

In [ ]:
all_result=[]
for i,row in enumerate(all): #row=[black pepper, water]    
    row_result=[]
    for phrase in row: #phrase=black pepper
        words=phrase.split(' ')
        for word in words: # black
            try:
                row_result.append(model[word][:20])
            except:
                continue
    row_result=np.mean(np.array(row_result),axis=0).tolist() 
    all_result.append(row_result)         

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
y = lb.fit_transform(train['cuisine'].tolist())

In [ ]:
x=all_result[:train.shape[0]]
temp=all_result[train.shape[0]:]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
model=RandomForestClassifier()
cross_val_score(model, x, y, cv=6, scoring = 'accuracy')

In [ ]:
model.fit(x,y)

In [ ]:
result=model.predict(temp)

In [ ]:
result_english=[lb.inverse_transform([i])[0] for i in result]

In [ ]:
submission_df=pd.Series(result_english,index=test.index).rename('cuisine')
submission_df.to_csv("logistic_sub.csv", index=True, header=True)